# Pràctica 2 de PLN: web scraping
En aquesta pràctica realitzarem diferents exercicis sobre captura de continguts web (web scraping) usant les llibreries `request` i `BeautifulSoup`.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Part 1: captura d'una notícia
En aquesta part ens descarregarem una notícia d'una pàgina web i desarem el contingut en disc.

In [184]:
url = "https://www.microsiervos.com/archivo/peliculas-tv/dia-de-la-marmota.html"
try:
    page = requests.get(url)
except:
    print("Error en obrir la URL")

### Exercici 1
Processa l'HTML capturat usant BeautifulSoup i guarda'l a la variable `soup`.

In [185]:
soup = BeautifulSoup(page.text)

El contingut de l'article (text del cos) es troba dins d'un Tag `<article>` de la pàgina (pots inspeccionar l'estructura HTML de la pàgina al navegador per comprovar-ho).
### Exercici 2
Carregueu el contingut de l'article en un objecte Tag de nom `article`.

In [192]:
article = soup.article
article.find('div',class_="autor").text.strip()

'Por @Alvy —  2 de Febrero de 2024'

El títol i l'autor de l'article estan en `<div>` d'una classe específica, i el text en una sèrie de paràgrafs. Al final hi ha una llista `<ul>` d'articles relacionats.
### Exercici 3
Obtingues el títol i l'autor en les variables `titul` i `autor` respectivament (com a objectes `NavigableString` de la llibreria BS).

In [24]:
titul = soup.find('div', class_='title').h1.a.string
autor = soup.find('div', class_='autor').a.string
print(f'Titulo: {titul}')
print(f'Autor: {autor}')

Titulo: Hoy es el día de la marmota
Autor: @Alvy


### Exercici 4
Extrau tot el contingut de text de l'article en una variable anomenada `contingut`.

Nota: haureu d'extraure el text de tots els elements `<p>` de l'article i concatenar-los en un únic *string*. Elimina els paràgrafs que no contenen cap text i l'últim paràgraf (conté el text *Relacionado*).

In [35]:
contenido = ""
for p in soup.find_all('p'):
    linea = p.get_text(strip=True)
    if "Relacionado" in linea:
        break
    if linea:
        contenido += linea
contenido

'¡Hoy es el día!El día de la marmota, 2 de febrero. Uno de los más curiosos,fluzeantesy divertidos, inmortalizado en la entrañable película de Bill Murray,Groundhog Day. Su título en España fueAtrapado en el tiempo(1993, Harold Ramis) aunque todo el mundo la conoce simplemente comoEl Día de la Marmota.Película de culto como ninguna,de esas que podrías ver quince o veinte veces sin problemas: está en el imaginario popular, algunas cadenas de televisión la emiten si parar durante las 24 horas e incluso se ha colado «oficialmente» en los rankings de las mejores películas «de culto» de las últimas décadas.Por alguna extraña razón es un largometraje especialmente popular entre losgeeks(!?) Tal vez sea por esa sensación dedéjà vuperpetuo, el constante«viaje en el tiempo»–aunque no se desvela nunca el mecanismo subyacente– su fino humor inteligente o todo a la vez.¿Una demostración del alcance de esos fans?Según los que la han revisado una y otra vez el protagonistapasa más de 33 años atrapad

### Exercici 5
Crea una funció per tornar només els paràgrafs que contenen algun text emfatitzat (tag `<strong>`) i executa-la per obtenir la llista de paràgrafs filtrats. Quants paràgrafs hi ha?

In [48]:
def obtenir_paragrafs_emfatitzats(html):
    paragrafs_filtrats = []

    for paragraf in soup.find_all('p'):
        if paragraf.find('strong'):
            paragrafs_filtrats.append(paragraf.get_text(strip=True))

    return paragrafs_filtrats

paragrafs_filtrats = obtenir_paragrafs_emfatitzats(article)
for i, paragraf in enumerate(paragrafs_filtrats, 1):
    print(f"Paràgraf {i}: {paragraf}")

nombre_paragrafs = len(paragrafs_filtrats)
print(f"\nNombre de paràgrafs filtrats: {nombre_paragrafs}")

Paràgraf 1: ¡Hoy es el día!El día de la marmota, 2 de febrero. Uno de los más curiosos,fluzeantesy divertidos, inmortalizado en la entrañable película de Bill Murray,Groundhog Day. Su título en España fueAtrapado en el tiempo(1993, Harold Ramis) aunque todo el mundo la conoce simplemente comoEl Día de la Marmota.
Paràgraf 2: Película de culto como ninguna,de esas que podrías ver quince o veinte veces sin problemas: está en el imaginario popular, algunas cadenas de televisión la emiten si parar durante las 24 horas e incluso se ha colado «oficialmente» en los rankings de las mejores películas «de culto» de las últimas décadas.
Paràgraf 3: Por alguna extraña razón es un largometraje especialmente popular entre losgeeks(!?) Tal vez sea por esa sensación dedéjà vuperpetuo, el constante«viaje en el tiempo»–aunque no se desvela nunca el mecanismo subyacente– su fino humor inteligente o todo a la vez.¿Una demostración del alcance de esos fans?Según los que la han revisado una y otra vez el pr

## Part 2: captura de dades meteorològiques
En aquesta part capturarem dades meteorològiques de la Comunitat Valenciana des de la pàgina de la [AVAMET (Associació valenciana de meteorologia)](https://www.avamet.org), dins de `meteoXarxa > MX Dades històriques`.
La URL de les dades meteorològiques accepta com a arguments GET la data i el territori.

### Exercici 6
Obtingues l'HTML de les dades històriques per al 1/2/2024 (`id='2024-02-01'`) al Camp de Túria (`territori='c11'`) i guarda l'HTML processat com un objecte `soup`.

In [58]:
url = "https://www.avamet.org/mx-meteoxarxa.php?id=2024-02-01&territori=c11"
try:
    page = requests.get(url)
except:
    print("Error en obrir la URL")
soup = BeautifulSoup(page.text)

Les dades de tots els municipis obtinguts estan en una taula de classe `tDades`.

In [74]:
tdatos = soup.find('table', class_="tDades")
tdatos

<table class="tDades" style="width: 98% !important;">
<thead class="fixe1">
<tr>
<th rowspan="2" width="35%">Estació</th>
<th colspan="3">Temperatura (°C)</th>
<th>HR (%)</th>
<th rowspan="2" width="10%">Prec. (mm)</th>
<th colspan="3">Vent</th>
</tr>
<tr>
<th width="7%">Mín</th>
<th width="7%">Mit</th>
<th width="7%">Màx</th>
<th width="7%">Mit</th>
<th width="8%">Mit</th>
<th width="8%">Dir</th>
<th width="8%">Màx</th>
</tr>
</thead>
<tr>
<td class="rProvincia" colspan="8">Província de València</td>
</tr>
<tr>
<td class="rComarca" colspan="8">el Camp de Túria</td>
</tr>
<tr>
<td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c11m051e01"><img alt="Estació associada a AVAMET" height="13" src="imatges/2017/clas/estrela-mx-soci.png" title="Estació associada a AVAMET" width="13"> Benaguasil<span class="rEstaDmxo"><span class="ptda"></span></span></img></a> </td>
<td class="rValm colornT17">5,7 </td>
<td class="rValm colornT20">11,7 </td>
<td class="rValm colornT25">20,1 </td>
<td cl

Dins de la taula, les dades estan a les files (`<tr>`) que tenen un element`<td class='rEsta'>`. Definim una funció per filtrar etiquetes `<tr>` el contingut de les quals tinga aquesta classe i llistem tots els elements interns a la taula.

In [84]:
def listar(tag):
    return tag.name == "tr" and tag.find(class_="rEsta")
loc = tdatos.find_all(listar)
loc

[<tr>
 <td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c11m051e01"><img alt="Estació associada a AVAMET" height="13" src="imatges/2017/clas/estrela-mx-soci.png" title="Estació associada a AVAMET" width="13"> Benaguasil<span class="rEstaDmxo"><span class="ptda"></span></span></img></a> </td>
 <td class="rValm colornT17">5,7 </td>
 <td class="rValm colornT20">11,7 </td>
 <td class="rValm colornT25">20,1 </td>
 <td class="rVal">68 </td>
 <td class="rValm colorP">0,0 </td>
 <td class="rVal">1,9 </td>
 <td class="rVal">E </td>
 <td class="rVal"><b>16,6 </b></td>
 </tr>,
 <tr>
 <td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c11m070e01"><img alt="Estació col·laboradora d'AVAMET" height="13" src="imatges/2017/clas/estrela-mx-cola.png" title="Estació col·laboradora d'AVAMET" width="13"> Bétera<span class="rEstaDmxo"><span class="ptda"></span>Mas de Juan Luís <img alt="IVIA" class="mxlogis" src="https://www.avamet.org/fotos/logis/logo-ivia.png"/></span></img></a> </td>
 <td cl

Veiem que algunes de les cel·les d'aquesta fila tenen elements de tipus `<span>`. Si no ens interessen els podríem eliminar amb el mètode `.decompose()` del Tag. Però com que el text ens interessa inserim un espai per separar el contingut en extraure el text posteriorment amb `.text()`. La `class` associada als `span` és `<span class='rEstaDmxo'>`.

In [125]:
lista = list()
for i in loc:
    i.find_all('span', class_="rEstaDmxo")
    i.insert_after(" ")
    i.insert_before(" ")
    lista.append(i)
#lista = [i.find_all('span', class_="rEstaDmxo") for i in loc]
lista 

[<tr>
 <td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c11m051e01"><img alt="Estació associada a AVAMET" height="13" src="imatges/2017/clas/estrela-mx-soci.png" title="Estació associada a AVAMET" width="13"> Benaguasil<span class="rEstaDmxo"><span class="ptda"></span></span></img></a> </td>
 <td class="rValm colornT17">5,7 </td>
 <td class="rValm colornT20">11,7 </td>
 <td class="rValm colornT25">20,1 </td>
 <td class="rVal">68 </td>
 <td class="rValm colorP">0,0 </td>
 <td class="rVal">1,9 </td>
 <td class="rVal">E </td>
 <td class="rVal"><b>16,6 </b></td>
 </tr>,
 <tr>
 <td class="rEsta"><a class="negre" href="mx-fitxa.php?id=c11m070e01"><img alt="Estació col·laboradora d'AVAMET" height="13" src="imatges/2017/clas/estrela-mx-cola.png" title="Estació col·laboradora d'AVAMET" width="13"> Bétera<span class="rEstaDmxo"><span class="ptda"></span>Mas de Juan Luís <img alt="IVIA" class="mxlogis" src="https://www.avamet.org/fotos/logis/logo-ivia.png"/></span></img></a> </td>
 <td cl

Cada cel·la `<td>` dins de la fila és una columna de la taula.
### Exercici 7
Obtingueu la llista de valors corresponents a la primera fila de la taula.

In [143]:
def funcion(tag):
    return (tag.name == "td" and ("rValm" in tag.get("class", []))) or \
           (tag.name == "td" and ("rVal" in tag.get("class", [])))
lista = loc[0].find_all(funcion)

print([i.text for i in lista])

['5,7 ', '11,7 ', '20,1 ', '68 ', '0,0 ', '1,9 ', 'E ', '16,6 ']


Captura totes les dades de la taula (llista de strings per cada fila de la taula) a l'objecte `dades`.

Nota: Utilitzeu una doble `comprehension` (o un bucle doble). La més interna recorrerà sobre les columnes i la externa sobre cada fila. Ajuda't del codi anterior.

In [144]:
[t.text.strip() for t in loc[0].find_all("td")]

['Benaguasil', '5,7', '11,7', '20,1', '68', '0,0', '1,9', 'E', '16,6']

Ara exportem la taula com un DataFrame de `pandas`. Les columnes, per ordre, han de ser: `Localitat`, `Temp`, `Tmax`, `Tmin`, `Humitat`, `Precipitacio`, `Vel_vent`, `Dir_vent`, `Vmax_vent`.

In [146]:
datos = [[t.text.strip() for t in l.find_all("td")] for l in loc]
datos

[['Benaguasil', '5,7', '11,7', '20,1', '68', '0,0', '1,9', 'E', '16,6'],
 ['BéteraMas de Juan Luís',
  '2,8',
  '9,3',
  '19,1',
  '78',
  '0,2',
  '2,7',
  'O',
  '11,2'],
 ['Béterala Conarda', '5,2', '11,2', '18,9', '', '0,2', '0,6', 'OSO', '12,9'],
 ['Bétera', '4,4', '10,8', '19,5', '74', '0,4', '0,5', 'O', '12,9'],
 ['BéteraUrbanització Mas Camarena',
  '5,5',
  '11,1',
  '19,4',
  '71',
  '0,0',
  '3,7',
  'SO',
  '11,4'],
 ['l′Elianacentre', '4,5', '10,3', '18,5', '70', '0,0', '2,7', 'ONO', '16,1'],
 ['l′ElianaBarranc de Mandor',
  '1,5',
  '8,3',
  '19,2',
  '64',
  '0,0',
  '1,4',
  'S',
  '13,3'],
 ['l′Elianapoble', '4,0', '11,2', '18,6', '', '0,0', '', 'NO', '16,0'],
 ['l′ElianaPinaeta del Cel',
  '3,8',
  '9,7',
  '18,6',
  '77',
  '0,0',
  '0,2',
  'NE',
  '13,0'],
 ['LlíriaBasseta del Vaquer',
  '1,3',
  '9,1',
  '18,8',
  '76',
  '0,2',
  '1,8',
  'N',
  '12,1'],
 ['Llíria', '3,4', '10,1', '19,5', '74', '0,0', '4,2', 'ONO', '12,9'],
 ['LoriguillaPozalet',
  '4,7',
  '10,7

### Exercici 8
Els codis de cada territori són a l'element `select` mostrat a continuació. Crea un dataframe amb les columnes `codi` i `nom` per a tots els territoris.

In [151]:
data = soup.find("select", attrs={'name':"freg_territori"})
terrirotios = data.find_all("option")
terrirotios

[<option style="font-weight:800;" value="pv">TOT EL TERRITORI</option>,
 <option style="font-weight:800;" value="p12">Prov. Castelló</option>,
 <option value="c01">   els Ports</option>,
 <option value="c02">   l′Alt Maestrat</option>,
 <option value="c03">   el Baix Maestrat</option>,
 <option value="c04">   l′Alcalatén</option>,
 <option value="c05">   la Plana Alta</option>,
 <option value="c06">   la Plana Baixa</option>,
 <option value="c07">   l′Alt Palància</option>,
 <option value="c08">   l′Alt Millars</option>,
 <option style="font-weight:800;" value="p46">Prov. València</option>,
 <option value="c09">   el Racó d′Ademús</option>,
 <option value="c10">   la Serrania</option>,
 <option selected="" value="c11">   el Camp de Túria</option>,
 <option value="c12">   el Camp de Morvedre</option>,
 <option value="c13">   l′Horta Nord</option>,
 <option value="c14">   l′Horta Oest</option>,
 <option value="c15">   València</option>,
 <option value="c16">   l′Horta Sud</option>,
 <opt

In [152]:
diccionario_municipios = {}

for opcion in terrirotios:
    valor = opcion['value'].strip()
    nombre = opcion.get_text(strip=True).replace(valor, '').strip()
    diccionario_municipios[valor] = nombre

diccionario_municipios

{'pv': 'TOT EL TERRITORI',
 'p12': 'Prov. Castelló',
 'c01': 'els Ports',
 'c02': 'l′Alt Maestrat',
 'c03': 'el Baix Maestrat',
 'c04': 'l′Alcalatén',
 'c05': 'la Plana Alta',
 'c06': 'la Plana Baixa',
 'c07': 'l′Alt Palància',
 'c08': 'l′Alt Millars',
 'p46': 'Prov. València',
 'c09': 'el Racó d′Ademús',
 'c10': 'la Serrania',
 'c11': 'el Camp de Túria',
 'c12': 'el Camp de Morvedre',
 'c13': 'l′Horta Nord',
 'c14': 'l′Horta Oest',
 'c15': 'València',
 'c16': 'l′Horta Sud',
 'c17': 'la Plana d′Utiel-Requena',
 'c18': 'la Foia de Bunyol',
 'c19': 'la Vall de Cofrents-Aiora',
 'c20': 'la Ribera Alta',
 'c21': 'la Ribera Baixa',
 'c22': 'la Canal de Navarrés',
 'c23': 'la Costera',
 'c24': 'la Vall d′Albaida',
 'c25': 'la Safor',
 'p03': 'Prov. Alacant',
 'c26': 'el Comtat',
 'c27': 'l′Alcoià',
 'c28': 'l′Alt Vinalopó',
 'c29': 'el Vinalopó Mitjà',
 'c30': 'la Marina Alta',
 'c31': 'la Marina Baixa',
 'c32': 'l′Alacantí',
 'c33': 'el Baix Vinalopó',
 'c34': 'el Baix Segura'}

## Part 3: informació de la Wikipedia
Les pàgines de la Wikipedia solen tenir una taula resum anomenada "Infobox" (normalment apareix a la dreta). Aquestes taules mostren parells de clau-valor amb el format:  
```
<tbody>
    <tr>
        <th>Clau</th>
        <td>Valor</td>
    </tr>
    ...
</tbody>
```


En aquesta part obtindrem les URL de les entrades a la Wikipedia per als premis Goya i obtindrem les seues dades bàsiques en forma de taula.\
El llistat dels premis es pot descarregar de la pàgina de la wikipedia següent:\
https://es.wikipedia.org/wiki/Anexo:XXXIV_edición_de_los_Premios_Goya

In [2]:
url ="https://es.wikipedia.org/wiki/Anexo:XXXIV_edici%C3%B3n_de_los_Premios_Goya"
try:
    page = requests.get(url)
except:
    print("Error en obrir la URL")
soup = BeautifulSoup(page.text)

### Exercici 9
Obtingueu una llista amb tots els tags `<tr>` (files) de la taula.

In [3]:
tr = soup.find_all('tr')
tr

[<tr><th class="cabecera" colspan="3" style="text-align:center;background-color:#FFEC80;color:inherit;">34.ª edición de los Premios Goya</th></tr>,
 <tr><th scope="row" style="text-align:left;width: 36%;;">Otorgado por</th><td colspan="2">
 <a href="/wiki/Academia_de_las_Artes_y_las_Ciencias_Cinematogr%C3%A1ficas_de_Espa%C3%B1a" title="Academia de las Artes y las Ciencias Cinematográficas de España">Academia de las Artes y las Ciencias Cinematográficas de España</a></td></tr>,
 <tr><th scope="row" style="text-align:left;width: 36%;;">Fecha</th><td colspan="2">
 25 de enero de 2020</td></tr>,
 <tr><th scope="row" style="text-align:left;width: 36%;;">Ubicación</th><td colspan="2">
 <a href="/wiki/M%C3%A1laga" title="Málaga">Málaga</a> y <a href="/wiki/Palacio_de_Deportes_Jos%C3%A9_Mar%C3%ADa_Mart%C3%ADn_Carpena" title="Palacio de Deportes José María Martín Carpena">Palacio Deportes José María Martín Carpena</a><br/><span style="display:none;">España</span><span class="flagicon"><span cla

Recorre totes les files de la taula i genera una llista de tuples clau-valor amb aquelles columnes que contenten les dues dades (`th` i `td` seran la clau i el valor que hem de buscar, respectivament).

In [19]:
diccionario_goya = []
for elem in tr:
     th = elem.find('th')
     td = elem.find('td')
     
     if td and th:
           th = th.get_text(strip=True)
           td = td.get_text(strip=True)
           diccionario_goya.append((th, td))

diccionario_goya

[('Otorgado por',
  'Academia de las Artes y las Ciencias Cinematográficas de España'),
 ('Fecha', '25 de enero de 2020'),
 ('Ubicación',
  'MálagayPalacio Deportes José María Martín CarpenaEspañaEspaña'),
 ('Anfitrión', 'Silvia AbrilyAndreu Buenamente')]

Converteix aquesta informació a un DataFrame amb les columnes clau-valor.

In [20]:
df = pd.DataFrame(diccionario_goya, columns=['Clave','Valor'])
df

,Clave,Valor
0,Otorgado por,Academia de las Artes y las Ciencias Cinematog...
1,Fecha,25 de enero de 2020
2,Ubicación,MálagayPalacio Deportes José María Martín Carp...
3,Anfitrión,Silvia AbrilyAndreu Buenamente


### Exercici 10
Recorre les 38 pàgines de tots els premis Goya i genera un dataframe amb les dades de data, ubicació i amfitrions de cada edició obtingudes a partir del `infobox` de cada entrada. Els `div` que ens interessen tenen el id `mw-pages`.

In [3]:
url ="https://es.wikipedia.org/wiki/Categor%C3%ADa:Anexos:Ediciones_de_los_Premios_Goya"
try:
    page = requests.get(url)
except:
    print("Error en obrir la URL")
soup = BeautifulSoup(page.text)

In [4]:
urls = [ item.a["href"] for item in soup.find("div", id= "mw-pages").ul.find_all("li") ]
url = "https://es.wikipedia.org"
urls = [url+i for i in urls]
urls

['https://es.wikipedia.org/wiki/Anexo:I_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:II_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:III_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:IV_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:V_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:VI_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:VII_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:VIII_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:IX_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:X_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:XI_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:XII_edici%C3%B3n_de_los_Premios_Goya',
 'https://es.wikipedia.org/wiki/Anexo:XIII_edici%C3%B3n_de_los_Premios_Goya',
 'https:

In [7]:
try:
    page = requests.get(urls[0])
except:
    print("Error en obrir la URL")
soup = BeautifulSoup(page.text)

lista = []
for u in urls:
    # Carga de la p'agina
    try:
        page = requests.get(urls[0])
    except:
        print("Error en obrir la URL")
    soup = BeautifulSoup(page.text)
    # Una vez cargada sacamos la infobox y la almacenamos en formato tupla en una lista
    infobox = soup.find('table', class_='infobox').find_all('td')

    lista_aux = []
    for td in infobox: # si la encontramos,
        td_colspan = td.get('colspan')
        if td_colspan != 3 and td_colspan != None:
            lista_aux.append(td.get_text(strip=True)) # Lo guardamos en una lista auxiliar
        lista.append(lista_aux) # Guardamos la auxiliar en la principal <- Guardamos la informacion de la gala en la lista
    break
lista

[['Academia de las Artes y las Ciencias Cinematográficas de España',
  '17 de marzo de 1987',
  'Teatro Lope de Vega (Madrid)EspañaEspaña',
  'Fernando Rey',
  'CronologíaPremios GoyaII edición',
  '[editar datos en Wikidata]'],
 ['Academia de las Artes y las Ciencias Cinematográficas de España',
  '17 de marzo de 1987',
  'Teatro Lope de Vega (Madrid)EspañaEspaña',
  'Fernando Rey',
  'CronologíaPremios GoyaII edición',
  '[editar datos en Wikidata]'],
 ['Academia de las Artes y las Ciencias Cinematográficas de España',
  '17 de marzo de 1987',
  'Teatro Lope de Vega (Madrid)EspañaEspaña',
  'Fernando Rey',
  'CronologíaPremios GoyaII edición',
  '[editar datos en Wikidata]'],
 ['Academia de las Artes y las Ciencias Cinematográficas de España',
  '17 de marzo de 1987',
  'Teatro Lope de Vega (Madrid)EspañaEspaña',
  'Fernando Rey',
  'CronologíaPremios GoyaII edición',
  '[editar datos en Wikidata]'],
 ['Academia de las Artes y las Ciencias Cinematográficas de España',
  '17 de marzo 